<a href="https://colab.research.google.com/github/RenatoCoronado99/Machine-Learning/blob/main/LogisticRegression_SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import math
import nltk
import csv
 
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
 
 
##Diccionario que tiene dividido las palabras sustantivos , verbos , adverbios para Lematizar el texto
from collections import defaultdict
from nltk.corpus import wordnet as wn
 
##Lematizar es convetir la palabra en su forma primitiva
from nltk.stem import WordNetLemmatizer
##Parts of speech = partes de laspalabras que indica el significado de cada una de las palabras (pronombre , sustantivos , etc )
from nltk import pos_tag
 
from nltk.corpus import stopwords
 
##Se parar las palabras entre una lista seaprada por comas 
from nltk import word_tokenize
 
from sklearn import model_selection,svm
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
 
## Para convertir las palabras a su forma raiz
from sklearn.feature_extraction.text import TfidfVectorizer
## Metrica que sirve para depurar los terminos en los textos
##Aquellas palabras que se repiten mas en los textos
#TF = medida que permite extraer palabras no comunes 
##ID = frecuencia inverse .... lo que dan mas información
from sklearn.metrics import accuracy_score,recall_score,f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

In [ ]:
file= pd.read_csv("test.csv",encoding='latin-1',names=["text","value"])
#file.drop(['Unnamed: 2','Unnamed: 3'], axis='columns', inplace=True)
file.drop(file.index[0:1], inplace=True)
print(file.dtypes)
print(file.head())
print(file.size)

In [ ]:
print("============================================ Seleccion de caracteristicas ===============================================")
##Se pone todo el texto a minuscula
for i in file["text"]:
  file["text"]=file["text"].replace(i,i.lower())
  #print(file["text"])
  ##Se para las palabras en un lista##
file["text"] = [word_tokenize(item) for item in file["text"]] 
#print(file["text"])
tag_map =defaultdict(lambda:wn.NOUN)##Cada palabra q no se enceuntre que es se tomara como un sustantivo
tag_map['J']=wn.ADJ
tag_map['V']=wn.VERB
tag_map['R']=wn.ADV
##steaming = cortar las palabras 
for index, item in enumerate(file["text"]):
  f_words = []## Colocar los textos que estan limpios (palabras que no tienen signos de puntuacióón , fechas )
  word_Lemmatized = WordNetLemmatizer() ## Lematizador = cortar las palabras con criterio de clasificar si es sustantivo, adjetivo ,verbo , etc 
  ##CLasificación de palabras
  for word,tag in pos_tag(item):##Parts of speech = partes de laspalabras que indica el significado de cada una de las palabras (pronombre , sustantivos , etc )
    #print(tag)
    ##Stop word = palabras que no brindan informacióón o  tienen simbolos de puntuación
    if word not in stopwords.words("english") and word.isalpha():##palabras no son numeros y no son stop words 
      f_words.append(word_Lemmatized.lemmatize(word,tag_map[tag[0]]))  ## Ayudando al Lematizador para que clasifique las palabras dandole como parametro la primera letra del tipo de palabra
  file.loc[index,'text_final'] = str(f_words)
#print(file.isnull().sum())
file = file.dropna()
#print(file.isnull().sum())
#print(file.head())

In [ ]:
Xtrain,Xtest,ytrain,ytest=model_selection.train_test_split(file["text_final"],file['value'],test_size=0.3)


In [ ]:
tfidf_vector = TfidfVectorizer(max_features= 50000)
tfidf_vector.fit(file["text_final"].astype(str))
Xtrain_tfidf=tfidf_vector.transform(Xtrain)
Xtest_tfidf=tfidf_vector.transform(Xtest)

In [ ]:
enc=LabelEncoder()
train_y=enc.fit_transform(ytrain)
test_y = enc.fit_transform(ytest)

In [ ]:
  
  print("============================================ GridSearch CV ===============================================")
  from sklearn.model_selection import GridSearchCV
  p=0
  model_params={
      "svm":{
          "model" : svm.SVC(),
          "params": {
              "C":[1,10,100],
              "kernel":["rbf","linear"],
              "gamma": [0.001,0.0001]
          }
      },
      "logistic_regression":{
          "model": LogisticRegression(),
          "params":{
              "C":[1,10,100],
              "penalty":["l1","l2"]
          }
      }
  }
  scores= []
  for model_name,mp in model_params.items():
    p=p+1
    print( str(mp["params"])+"==========>"+str(p))
    clf = GridSearchCV(mp["model"],mp["params"],cv=5,n_jobs=-1)
    clf.fit(Xtrain_tfidf,train_y)
    scores.append({
        "model":model_name,
        "best_score":clf.best_score_,
        "best_params":clf.best_params_,
        "best_estimator":clf.best_estimator_
    })
  print("================================ Comparing_SVM_RLogistic ========================")
  df = pd.DataFrame(scores,columns=["model","best_score","best_params"])
  print(df)

In [ ]:

best_params_svm= df.iat[0,2]
best_params_logistic_regression= df.iat[1,2]
print(best_params_svm)
print(best_params_logistic_regression)


In [ ]:
#print(svmc.predict(Xtest_tfidf))
#print(Xtest_tfidf)

In [ ]:
tabla = []
print("====================== SVM ==============================")

#print(Xtrain_tfidf)
enc=LabelEncoder()

svmc=svm.SVC(C=best_params_svm["C"],kernel=best_params_svm["kernel"],gamma=best_params_svm["gamma"])
svmc.fit(Xtrain_tfidf,train_y)
pred = svmc.predict(Xtest_tfidf)
print(pred)
print("#####################Accurary_score##################")
acc = accuracy_score(pred,test_y)
print(acc*100)
print("#####################Matrix de Confusión##################")
cm = confusion_matrix(pred,test_y)
print(cm)
print("##################### F1 Score ##################")
f1 = f1_score(pred,test_y)
print("f1 Score -> ", f1)
print("##################### RECALL ##################")
recall  = recall_score(pred,test_y)
print("El modelo solo es capaz de identificar el %0.5f " %recall)

tabla.append({
    "acc": acc,
    "conf_mat" : cm,
    "F1" : f1,
    "recall":recall
})


In [ ]:
print("====================== Logistic Regression ==============================")

r_logs= LogisticRegression(C=best_params_logistic_regression["C"],penalty=best_params_logistic_regression["penalty"])
r_logs= LogisticRegression(C=1.0,penalty="l2")
r_logs.fit(Xtrain_tfidf,train_y)
y_pred_rlogs = r_logs.predict(Xtest_tfidf)
print(y_pred_rlogs)
print("#####################Accurary_score##################")
acc = accuracy_score(y_pred_rlogs,test_y)
print(acc)
print("#####################Matrix de Confusión##################")
cm = confusion_matrix(y_pred_rlogs,test_y)
print(cm)
print("##################### F1 Score ##################")
f1 = f1_score(y_pred_rlogs,test_y)
print("f1 Score -> ", f1)
print("##################### RECALL ##################")
recall  = recall_score(y_pred_rlogs,test_y)
print("El modelo solo es capaz de identificar el %0.5f " %recall)
tabla.append({
    "acc": acc,
    "conf_mat" : cm,
    "F1" : f1,
    "recall":recall
})

In [ ]:
table_dataframe = pd.DataFrame(tabla,columns=["acc","F1","recall"])
print(table_dataframe)